# Managing Agent State - Saving and Loading Agents

In [2]:
import asyncio
from autogen_core import CancellationToken
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)

In [3]:
assistant = AssistantAgent(
    name = 'Assistant',
    model_client = model_client,
    system_message = 'You are a helpful assistant who can answer questions and provide information on a wide range of topics. You are friendly, informative, and concise.'
)

response = await assistant.on_messages(
    [TextMessage(content = 'Can you help me with a travel itinerary of France from May to June? Keep it under 30 words.',
    source = 'user')],
    cancellation_token = CancellationToken(),
)

print(response.chat_message)

source='Assistant' models_usage=RequestUsage(prompt_tokens=62, completion_tokens=37) metadata={} content="Explore Paris, then visit the Loire Valley for châteaux. Head to Bordeaux for wine tasting, drive through Provence, and relax on the Côte d'Azur before ending in Nice." type='TextMessage'


In [4]:
agent_state = await assistant.save_state()

In [8]:
print(agent_state)

{'type': 'AssistantAgentState', 'version': '1.0.0', 'llm_context': {'messages': [{'content': 'Can you help me with a travel itinerary of France from May to June? Keep it under 30 words.', 'source': 'user', 'type': 'UserMessage'}, {'content': "Explore Paris, visit the Loire Valley, tour the French Riviera, discover Provence, experience Bordeaux's wine region, and explore Normandy's history. Allow time for spontaneous exploration and relaxation.", 'thought': None, 'source': 'Assistant', 'type': 'AssistantMessage'}]}}


In [9]:
new_assistant_agent = AssistantAgent(
    name = 'Assistant_agent_2',
    model_client = model_client,
    system_message = 'You are a helpful assistant.'
)

In [10]:
await new_assistant_agent.load_state(agent_state)

In [11]:

response = await new_assistant_agent.on_messages(
    [TextMessage(content = 'Can you tell me what was the last discussion about?',
    source = 'user')],
    cancellation_token = CancellationToken(),
)
print(response.chat_message)

source='Assistant_agent_2' models_usage=RequestUsage(prompt_tokens=98, completion_tokens=53) metadata={} content='You asked me to help create a travel itinerary for France from May to June, keeping it under 30 words. I provided a brief itinerary highlighting key regions to explore, including Paris, the Loire Valley, the French Riviera, Provence, Bordeaux, and Normandy.' type='TextMessage'


In [12]:
response.chat_message.content

'You asked me to help create a travel itinerary for France from May to June, keeping it under 30 words. I provided a brief itinerary highlighting key regions to explore, including Paris, the Loire Valley, the French Riviera, Provence, Bordeaux, and Normandy.'